## Nearest-edge Map Matching

#### = 各GPS測位点を最寄りのエッジ（道路リンク）に割り当てるマッチングアルゴリズム

- 位置情報データ: df

- ある一人のユーザの位置情報データ: df_1user

In [ ]:
import pandas as pd

df = pd.read_csv("位置情報csvのパス") #パスを記入
df_1user = df[df["userid"]=="調べたいユーザのID"] #ユーザIDを記入

In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point

# -----------------------------
# 1) OSMグラフの準備（中区などポリゴンで取得）
# -----------------------------
place = "Naka Ward, Yokohama, Kanagawa, Japan"
gdf_place = ox.geocode_to_gdf(place)
polygon = gdf_place.loc[0, "geometry"]

G = ox.graph_from_polygon(polygon, network_type="all", simplify=True)
G_proj = ox.project_graph(G)
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_proj)

# -----------------------------
# 2) ユーザーGPSデータをGeoDataFrame化
# df_1user: lon, lat, recordedat列を持つ DataFrame
# -----------------------------
# recordedat を datetime 型に変換
df_1user["recordedat"] = pd.to_datetime(df_1user["recordedat"])

gdf_user = gpd.GeoDataFrame(
    df_1user,
    geometry=gpd.points_from_xy(df_1user.lon, df_1user.lat),
    crs="EPSG:4326"
)
gdf_user = gdf_user.to_crs(G_proj.graph['crs'])
gdf_user = gdf_user.sort_values("recordedat").reset_index(drop=True)

In [ ]:
from shapely.ops import nearest_points

# -----------------------------
# 3) 最近傍エッジにスナップ
# -----------------------------
# osmnxのnearest_edgesを使う（内部はKDTreeで効率的）
x = gdf_user.geometry.x.values
y = gdf_user.geometry.y.values
nearest_edges = ox.distance.nearest_edges(G_proj, X=x, Y=y)

gdf_user["nearest_edge"] = nearest_edges

# -----------------------------
# 4) スナップ点を計算
# -----------------------------
snap_points = []
for pt, edge in zip(gdf_user.geometry, gdf_user["nearest_edge"]):
    # edgeは(u, v, key) のタプル
    edge_geom = edges_gdf.loc[edge, "geometry"]
    nearest_pt = nearest_points(pt, edge_geom)[1]
    snap_points.append(nearest_pt)

gdf_user["snap_point"] = snap_points

# -----------------------------
# 5) 可視化
# -----------------------------
gps_x = gdf_user.geometry.x.values
gps_y = gdf_user.geometry.y.values
snap_x = [p.x for p in gdf_user["snap_point"]]
snap_y = [p.y for p in gdf_user["snap_point"]]

fig, ax = plt.subplots(figsize=(10,10))
ox.plot_graph(G_proj, ax=ax, show=False, close=False, node_size=0, edge_color="lightgray")
ax.plot(gps_x, gps_y, "o-", color="blue", alpha=0.6, label="Original GPS")
ax.plot(snap_x, snap_y, "o-", color="red", alpha=0.6, label="Snapped to Road")
ax.legend(fontsize=12)
plt.title("GPS vs Nearest-Edge Map Matching")
plt.show()